<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
    <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
    <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6"></ul>
        <li>Recreate the customer support ticket workflow using an <b>evaluator-optimizer agentic workflow pattern</b> instead of prompt chaining.</li>
        <li>Your evaluator agent should assess the quality and completeness of each ticket and suggest improvements.</li>
        <li>Your optimizer agent should revise the tickets based on evaluator feedback, aiming for clarity and actionable details.</li>
        <li>Try to implement this using at least two LLM calls (one for evaluation, one for optimization) and display the before/after results.</li>
        <li>Share your work in the community-contributions folder by creating a folder with your name. Eg. shaheer-airaj.</li>
    </ul>
    <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not loaded.")
if not ANTHROPIC_API_KEY:
    raise ValueError("ANTHROPIC_API_KEY not loaded.")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not loaded.")

In [3]:
# create a ticket
openai_client = OpenAI(api_key= OPENAI_API_KEY)

message = "I want you to create a customer ticket, the ticket should be less than 100 words, and can be from a variety of topics. Only show the content of the ticket." 
response = openai_client.chat.completions.create(
    model="gpt-4.1-nano",
    messages = [{"role": "user", "content": message}]
)
ticket = response.choices[0].message.content
display(Markdown(f"### Generated Ticket:\n\n{ticket}"))

### Generated Ticket:

Subject: Issue with Recent Order Delivery

Hello, I received my order (Order #12345) today, but the items are incorrect. I requested a blue sweater in size M, but received a red one in size L. Please advise on how to return the incorrect item and get the correct one shipped. Thank you for your assistance.

In [4]:
# create an evaluator
evaluator_prompt = f"""You are an expert customer support ticket evaluator. Your task is to assess the quality of customer support ticket in triple backticks based on quality and completeness, and suggest improvements if necessary. ```{ticket}```. 
  Your output should only be in two sentences one for evaluation and one for feedback. Be concise and less than 100 words."""
print(evaluator_prompt)
anthropic_client = OpenAI(api_key=ANTHROPIC_API_KEY, base_url="https://api.anthropic.com/v1")
evaluation_response = anthropic_client.chat.completions.create(
    model="claude-3-5-haiku-latest",
    messages = [{'role':'user', 'content': evaluator_prompt }],
    max_tokens=100)
evaluation = evaluation_response.choices[0].message.content
display(Markdown(f"### Ticket Evaluation:\n\n{evaluation}"))

You are an expert customer support ticket evaluator. Your task is to assess the quality of customer support ticket in triple backticks based on quality and completeness, and suggest improvements if necessary. ```Subject: Issue with Recent Order Delivery

Hello, I received my order (Order #12345) today, but the items are incorrect. I requested a blue sweater in size M, but received a red one in size L. Please advise on how to return the incorrect item and get the correct one shipped. Thank you for your assistance.```. 
  Your output should only be in two sentences one for evaluation and one for feedback. Be concise and less than 100 words.


### Ticket Evaluation:

Evaluation: The ticket provides clear, specific details about the order discrepancy, including order number, expected vs. received item details, which enables efficient problem resolution. Feedback: To improve the ticket, the customer could include additional context such as the order date, preferred return method, and contact information to expedite the resolution process.

In [5]:
# create an optimizer to improve the ticket
optimizer_prompt = f"""You are an expert customer support ticket optimizer. Your task is to improve the following customer support ticket in triple backticks based on the evaluation and feedback provided. 
  Ticket: ```{ticket}```. Evaluation and Feedback: ```{evaluation}```. 
  Your output should only be the improved ticket content, be concise and less than 100 words."""

gemini_client = OpenAI(api_key=GEMINI_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
# do not use max_tokens for gemini
optimization_response = gemini_client.chat.completions.create(
    model= "gemini-2.5-flash",
    messages = [{'role':'user', 'content': optimizer_prompt }]
    )
optimized_ticket = optimization_response.choices[0].message.content
display(Markdown(f"### Original Ticket:\n\n{ticket}"))
display(Markdown(f"### Evaluation and Feedback:\n\n{evaluation}"))
display(Markdown(f"### Optimized Ticket:\n\n{optimized_ticket}"))

### Original Ticket:

Subject: Issue with Recent Order Delivery

Hello, I received my order (Order #12345) today, but the items are incorrect. I requested a blue sweater in size M, but received a red one in size L. Please advise on how to return the incorrect item and get the correct one shipped. Thank you for your assistance.

### Evaluation and Feedback:

Evaluation: The ticket provides clear, specific details about the order discrepancy, including order number, expected vs. received item details, which enables efficient problem resolution. Feedback: To improve the ticket, the customer could include additional context such as the order date, preferred return method, and contact information to expedite the resolution process.

### Optimized Ticket:

Subject: Incorrect Order Received - Order #12345 (Ordered Oct 26, 2023)

Hello,
My order #12345, placed on Oct 26, 2023, arrived today with incorrect items. I requested a blue sweater (size M) but received a red one (size L).

Please provide instructions for returning the incorrect item and shipping the correct one. I prefer a prepaid return label. You can reach me at [Your Contact Email/Phone].